In [1]:
!conda list opencv-python

# packages in environment at C:\Users\admin\anaconda3:
#
# Name                    Version                   Build  Channel


In [2]:
!pip install opencv-python

# 실습 환경 준비

In [3]:
import cv2
import os
import glob
import shutil
import random
import string
import numpy as np
import pandas as pd

In [4]:
PATH_DEFECT = 'dataset/Defect_images/'
PATH_MASK = 'dataset/Mask_images/'
PATH_NODEFECT = 'dataset/NODefect_images/'

### Dataset 불러오기

In [13]:
defect_list = glob.glob(PATH_DEFECT + '*.png')
mask_list = glob.glob(PATH_MASK + '*.png')
pass_list = glob.glob(PATH_NODEFECT + '*/*.png')
# defect_list[0]
# Match defect-mask pairs
new_defect_list = list()
new_mask_list = list()
for defect in defect_list:
    num = defect.split('\\')[-1].split('_')[0]
    for mask in mask_list:
        num_mask = mask.split('\\')[-1].split('_')[0]
        if num == num_mask:
            new_defect_list.append(defect)
            new_mask_list.append(mask)
            break
defect_list = new_defect_list
mask_list = new_mask_list

In [11]:
len(defect_list), len(mask_list)

(105, 105)

In [14]:
pass_list[0]

'dataset/NODefect_images\\2306881-210020u\\0001_000_05.png'

### 첫 발송 데이터 생성

In [19]:
if os.path.exists('dataset/1') is False:
    os.mkdir('dataset/1')
for file_name in pass_list + defect_list:
#     print(file_name)
    if random.randint(0, 9) < 2:
        barcode = ''.join(random.choices(string.ascii_letters + string.digits, k=16))
#         print(barcode)
        shutil.copy2(file_name, 'dataset/1/' + barcode + '.png')

### 두번째 발송 데이터 생성

In [ ]:
if os.path.exists('dataset/2') is False:
    os.mkdir('dataset/2')
if os.path.exists('dataset/2/OK') is False:
    os.mkdir('dataset/2/OK')
if os.path.exists('dataset/2/FAIL') is False:
    os.mkdir('dataset/2/FAIL')
idx = 0
for file_name in pass_list:
    img = cv2.imread(file_name)
    height, width, _ = img.shape
    step = height // 2
#     print("1=> ", step, height, width, width // step) 
    for i in range(width // step):
        w = i * step
#         print("2=> ", i, height, width, width // step, w)  
        if w < width - height and random.randint(0, 9) < 2:
#             print("3=> ", i, step, "시작값=>", w)
            patch = img[:, w:w+height, :]
            cv2.imwrite('dataset/2/OK/%04d.png'%idx, patch)
            idx += 1

patch_list = list()
for item in zip(defect_list, mask_list):
    defect, mask = item
    img_d = cv2.imread(defect)
    img_m = cv2.imread(mask)
    height, width, _ = img_d.shape
    step = height // 2
    for i in range(width // step):
        w = i * step
        if w < width - height:
            patch = img_d[:, w:w+height, :]
            patch_d = img_m[:, w:w+height, :]
            if patch_d.sum() > 0:
                patch_list.append(patch)
random.shuffle(patch_list)
patch_list_fraction = patch_list[:len(patch_list)//3]
for idx, patch in enumerate(patch_list_fraction):
    cv2.imwrite('dataset/2/FAIL/%04d.png'% idx, patch)

### 세번째 발송 데이터 생성

In [51]:
if os.path.exists('dataset/3') is False:
    os.mkdir('dataset/3')
if os.path.exists('dataset/3/OK') is False:
    os.mkdir('dataset/3/OK')
if os.path.exists('dataset/3/FAIL') is False:
    os.mkdir('dataset/3/FAIL')
if os.path.exists('dataset/3/MASK') is False:
    os.mkdir('dataset/3/MASK')
idx = 0
for file_name in pass_list:
    img = cv2.imread(file_name)
    height, width, _ = img.shape
    step = height // 2
    for i in range(width // step):
        w = i * step
        if w < width - height and random.randint(0, 9) < 3:
            patch = img[:, w:w+height, :]
            cv2.imwrite('dataset/3/OK/%04d.png'%idx, patch)
            idx += 1

patch_pair_list = list()
for item in zip(defect_list, mask_list):
    defect, mask = item
    img_d = cv2.imread(defect)
    img_m = cv2.imread(mask)
    
    height, width, _ = img_d.shape
    step = height // 2
    for i in range(width // step):
        w = i * step
        if w < width - height:
            patch = img_d[:, w:w+height, :]
            patch_m = img_m[:, w:w+height, :]
            if patch_m.sum() > 0:
                patch_pair_list.append((patch, patch_m))
random.shuffle(patch_pair_list)
for idx, pair in enumerate(patch_pair_list):
    patch, patch_m = pair
    cv2.imwrite('dataset/3/FAIL/%04d.png' % idx, patch)
    cv2.imwrite('dataset/3/MASK/%04d.png'% idx, patch_m)

### 실전 데이터 생성

In [52]:
if os.path.exists('data/') is False:
    os.mkdir('data')
if os.path.exists('tfrecords/') is False:
    os.mkdir('tfrecords')
if os.path.exists('model/') is False:
    os.mkdir('model')
if os.path.exists('data/input_data/') is False:
    os.mkdir('data/input_data')
if os.path.exists('data/output_csv/') is False:
    os.mkdir('data/output_csv')

idx = 0
for file_name in pass_list:
    img = cv2.imread(file_name)
    height, width, _ = img.shape
    step = height // 2
    for i in range(width // step):
        w = i * step
        if w < width - height and random.randint(0, 9) < 5:
            patch = img[:, w:w+height, :]
            cv2.imwrite('data/input_data/ok_%04d.png'%idx, patch)
            idx += 1

patch_pair_list = list()
for item in zip(defect_list, mask_list):
    defect, mask = item
    img_d = cv2.imread(defect)
    img_m = cv2.imread(mask)
    
    height, width, _ = img_d.shape
    step = height // 2
    for i in range(width // step):
        w = i * step
        if w < width - height:
            patch = img_d[:, w:w+height, :]
            patch_m = img_m[:, w:w+height, :]
            if patch_m.sum() > 0:
                patch_pair_list.append((patch, patch_m))
random.shuffle(patch_pair_list)
for idx, pair in enumerate(patch_pair_list):
    patch, patch_m = pair
    cv2.imwrite('data/input_data/fail_%04d.png' % idx, patch)
#     cv2.imwrite('dataset/3/MASK/%04d.png'% idx, patch_m)

In [64]:
len(defect_list), len(mask_list), len(patch_pair_list[0])

(105, 105, 2)

In [65]:
defect_list[0]

'dataset/Defect_images\\0001_002_00.png'